In [26]:
import math
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [5]:
import csv
import numpy as np
import pandas as pd
df = pd.read_csv('winequality-red.csv', delimiter = ';')
print(df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  
0      9.4        5  
1      9.8        5  
2      9.8        5 

In [6]:
df.loc[df['quality'] >5, 'quality'] = 'good'
df.loc[df['quality'] !='good', 'quality'] = 'bad'

In [7]:
print(df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol quality  
0      9.4     bad  
1      9.8     bad  
2      9.8     bad  
3 

In [14]:
len(df['quality'])

1599

In [16]:
ind = range(len(df['quality']))

In [19]:
print(ind)

[False  True  True ... False  True  True]


In [11]:
ind = range(len(df['quality']))

In [18]:
ind = np.array(ind)
ind = ind%4 != 0
print(ind)

[False  True  True ... False  True  True]


In [21]:
train = df[ind==True]
test = df[ind==False]
features = df.columns[:11]
print(features)

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol'],
      dtype='object')


In [23]:
train_x = train[features]
train_y = train['quality']
test_x = test[features]
test_y = test['quality']
print(train_x.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
5            7.4              0.66         0.00             1.8      0.075   
6            7.9              0.60         0.06             1.6      0.069   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
5                 13.0                  40.0   0.9978  3.51       0.56   
6                 15.0                  59.0   0.9964  3.30       0.46   

   alcohol  
1      9.8  
2      9.8  
3      9.8  
5      9.4  
6      9.4  


In [ ]:
def entropy(arr):
	classCounts = np.unique(arr, return_counts=True)[1]
	sampleNum = arr.shape[0]
	if sampleNum==0:
		return 0
	p_arr = classCounts / float(sampleNum)
	ret = -np.log(p_arr)*p_arr
	return sum(ret)

In [ ]:


def gini(arr):
	classCounts = np.unique(arr, return_counts=True)[1]
	sampleNum = arr.shape[0]
	p_arr = classCounts / float(sampleNum)
	ret = 1. - (p_arr**2).sum()
	return ret

def giniSplit(arr1, arr2):
	sampleNum1 = float(arr1.shape[0])
	sampleNum2 = float(arr2.shape[0])
	totalSampleNum = sampleNum1 + sampleNum2
#	return entropy(arr1) * sampleNum1 / totalSampleNum + \
	#	   entropy(arr2) * sampleNum2 / totalSampleNum
	return gini(arr1) * sampleNum1 / totalSampleNum + \
		   gini(arr2) * sampleNum2 / totalSampleNum


minGiniSplit = np.inf
minSplitFeature = None
minSplitBoundary = None
feature = train_x


for fIdx in range(feature.shape[1]):
	featureArr = train_x[train_x.columns[fIdx]]
	uniqueSorted = np.unique(np.sort(featureArr))
	for i in range(len(uniqueSorted)-1):
		splitBoundary = (uniqueSorted[i] + uniqueSorted[i+1]) / 2.
		split1 = train_y[featureArr <= splitBoundary]
		split2 = train_y[featureArr > splitBoundary]
		giniSplitValue = giniSplit(split1, split2)

		if giniSplitValue < minGiniSplit:
			minGiniSplit = giniSplitValue
			minSplitFeature = fIdx
			minSplitBoundary = splitBoundary


print 'minSplitValue:     {}'.format(minGiniSplit)
print 'minSplitFeature:   {}'.format(train_x.columns[minSplitFeature])
print 'minSplitBoundary:  {}'.format(minSplitBoundary)